In [3]:
# Amazon Food Review

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix 
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

con = sqlite3.connect('./food_review/database.sqlite')

# run the sql cmd using the connection
filtered_data = pd.read_sql_query("""
    select * from REVIEWS where Score != 3
""", con)

def partition(x):
    if x <3:
        return 'negative'
    return 'positive'

# change score with negative, positive
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative




In [4]:
filtered_data.shape
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
display = pd.read_sql_query("""
    select * from REVIEWS where Score != 3 and UserId = 'AR5J8UI46CURR'
    order by ProductId
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [6]:
sorted_data = filtered_data.sort_values('ProductId', axis = 0, ascending=True)

In [7]:
# Remove the duplicates
final = sorted_data.drop_duplicates(subset={"UserId", "ProfileName", "Time", "Text"}, keep='first', inplace= False)
final.shape

(364173, 10)

In [8]:
# check the % of data that still remains
(final['Id'].size * 1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [9]:
display = pd.read_sql_query("""
    select * from REVIEWS where Score != 3 and Id=44737 or Id=64422
    order by ProductId
""", con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [10]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape

(364171, 10)

In [11]:
final['Score'].value_counts()
final

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
...,...,...,...,...,...,...,...,...,...,...
178145,193174,B009RSR8HO,A4P6AN2L435PV,romarc,0,0,positive,1350432000,LOVE!! LOVE!!,"LOVE, LOVE this sweetener!! I use it in all m..."
173675,188389,B009SF0TN6,A1L0GWGRK4BYPT,Bety Robinson,0,0,positive,1350518400,Amazing!! Great sauce for everything!,You have to try this sauce to believe it! It s...
204727,221795,B009SR4OQ2,A32A6X5KCP7ARG,sicamar,1,1,positive,1350604800,Awesome Taste,I bought this Hazelnut Paste (Nocciola Spread)...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [12]:
# BOW

count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)


In [13]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [14]:
final_counts.shape

(364171, 115281)

In [15]:
import re;
i = 0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent ))):
        print(i)
        print(sent)
        break;
    i += 1;
    

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [16]:
# Text Pre-Processing
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop= set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]', r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
print(stop)
print("***************************")
print(sno.stem('tasty'))

{'just', 'those', 'doing', 'ours', 'into', 'that', 'can', 'aren', 'has', 'some', 'these', 'through', 'further', 'it', 'she', 'from', "that'll", 'own', "needn't", "wasn't", "didn't", 'how', "wouldn't", 'all', 'ma', 'why', 'about', 'on', 'himself', 'have', 'had', 'was', "weren't", 'he', 'who', 'hers', 'and', 'ain', 'yours', 'which', "you're", 'only', 'does', 'are', 'down', 'because', 'until', "should've", "mustn't", 'too', 'off', 'nor', "hasn't", 'ourselves', 'same', 'yourself', 'do', 'here', 'between', 'in', 'couldn', 'y', "haven't", 'there', 'they', 'while', 'both', "shouldn't", 'for', 've', 'been', 'by', 'doesn', "hadn't", 'this', 'our', 'i', 'a', 'an', "couldn't", 'under', 'mightn', 'you', 'myself', 'wasn', "she's", 'having', "aren't", 'weren', 'don', 'his', 'during', 'were', 'being', 'or', 'hasn', 'after', 'your', 'd', 'over', 'won', 'them', 'before', 'such', 'but', 'be', "you've", 'him', 'did', 'the', 'so', 'o', 'needn', "it's", 'most', 'few', 's', 'am', 'below', 'themselves', 'wit

In [17]:
i=0;
str1=''
final_string=[]
pos_words=[]
neg_words=[]
s=''
for s in final['Text'].values:
    f=[]
    s=cleanhtml(s)
    for w in s.split():
        for c in cleanpunc(w).split():
            if((c.isalpha())&(len(c)>2)):
                if(c.lower()not in stop):
                    s=(sno.stem(c.lower())).encode('utf8')
                    f.append(s)
                    if(final['Score'].values)[i]=='positive':
                        pos_words.append(s)
                    if(final['Score'].values)[i]=='negative':
                        neg_words.append(s)
                else:
                    continue
            else:
                continue
    str1=b"".join(f)
    final_string.append(str1)
    i=i+1

In [18]:
final['CleanedText']=final_string
conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')

In [19]:
freq_dist_positive = nltk.FreqDist(pos_words)
freq_dist_negative = nltk.FreqDist(neg_words)

print("Most common positive words: ",freq_dist_positive.most_common(20))
print("Most common negative words: ",freq_dist_negative.most_common(20))

Most common positive words:  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791), (b'tea', 83888), (b'coffe', 78814), (b'make', 75107), (b'get', 72125), (b'food', 64802), (b'would', 55568), (b'time', 55264), (b'buy', 54198), (b'realli', 52715), (b'eat', 52004)]
Most common negative words:  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716), (b'get', 13786), (b'buy', 13752), (b'order', 12871), (b'food', 12754), (b'dont', 11877), (b'tea', 11665), (b'even', 11085), (b'box', 10844), (b'amazon', 10073), (b'make', 9840)]


In [20]:
count_vect = CountVectorizer(ngram_range=(1,2))
final_bigram_count = count_vect.fit_transform(final['Text'].values)

In [21]:
final_bigram_count.get_shape()

(364171, 2910192)

In [22]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)
final_tf_idf.get_shape()

(364171, 2910192)

In [24]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [25]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [26]:
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [28]:
def top_tfidf_feats(row, features, top_n=25):
    """ Get top n tfidf values in row and return them with their corresponding values"""
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0], features, 25)

In [29]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


In [1]:
# !python -m pip install -U gensim

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

model = KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)
